In [1]:
from load_model import get_reid_embeddings

c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\bahawalkhan\Anaconda3\envs\yolo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using Transformer_type: vit_base_patch16_224_TransReID as a backbone
using stride: [14, 14], and part number is num_y18 * num_x18
using drop_path_rate is : 0.1
using aie_xishu is : 2.5
Loading finetune model......from model-weights/transformer_40.pth
===========building transformer===========
base.cls_token
base.pos_embed
base.patch_embed.proj.weight
base.patch_embed.proj.bias
base.blocks.0.norm1.weight
base.blocks.0.norm1.bias
base.blocks.0.attn.qkv.weight
base.blocks.0.attn.qkv.bias
base.blocks.0.attn.proj.weight
base.blocks.0.attn.proj.bias
base.blocks.0.norm2.weight
base.blocks.0.norm2.bias
base.blocks.0.mlp.fc1.weight
base.blocks.0.mlp.fc1.bias
base.blocks.0.mlp.fc2.weight
base.blocks.0.mlp.fc2.bias
base.blocks.1.norm1.weight
base.blocks.1.norm1.bias
base.blocks.1.attn.qkv.weight
base.blocks.1.attn.qkv.bias
base.blocks.1.attn.proj.weight
base.blocks.1.attn.proj.bias
base.blocks.1.norm2.weight
base.blocks.1.norm2.bias
base.blocks.1.mlp.fc1.weight
base.blocks.1.mlp.fc1.bias
base.blo

In [2]:
import os
import cv2
import numpy as np

In [5]:
base_dir = '../augmented_cars_data'
mode = 'test-2'

In [6]:
def get_cars(car_dir):
    return [f'{base_dir}/{x}' for x in os.listdir(car_dir) if (os.path.isdir(f"{car_dir}/{x}" )) ]


In [7]:
from tqdm import tqdm

In [8]:
import random
def get_image_list(car_dir):
    imgs = []
    for source_dir in get_cars(car_dir):
        source_dir_path = f'{car_dir}/{source_dir}'
        my_list = os.listdir(source_dir_path)
        imgs.extend([source_dir +'/'+ x for x in my_list])
    return imgs
img_lst = [x for x in get_image_list(base_dir)]

len(img_lst)
imgs = img_lst 
print(imgs[:5])

['../augmented_cars_data/car1 augmented/IMG_4029.MOV_1.jpg', '../augmented_cars_data/car1 augmented/IMG_4029.MOV_10.jpg', '../augmented_cars_data/car1 augmented/IMG_4029.MOV_100.jpg', '../augmented_cars_data/car1 augmented/IMG_4029.MOV_101.jpg', '../augmented_cars_data/car1 augmented/IMG_4029.MOV_102.jpg']


In [9]:
import os
import random
from itertools import combinations, product
from tqdm import tqdm
def generate_pairs(dir_list, num_intra_pairs_per_dir, num_inter_pairs):
    intra_pairs = []
    inter_pairs = []

    # Iterate through each directory
    for directory in tqdm(dir_list):
        # Get a list of all image files in the directory
        image_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(('.jpg', '.png', '.jpeg'))]
        # Generate intra-directory pairs
        # image_files = random.sample(image_files,10)
        combs = combinations(image_files, 2)
        for com in combs:
            intra_pairs.append([com[0],com[1],1])

    # Generate inter-directory pairs
    for _ in tqdm(range(len(intra_pairs))):
        # Randomly select two different directories
        dir1, dir2 = random.sample(dir_list, 2)

        # Randomly select an image from each directory
        image1 = random.choice([os.path.join(dir1, file) for file in os.listdir(dir1) if file.endswith(('.jpg', '.png', '.jpeg'))])
        image2 = random.choice([os.path.join(dir2, file) for file in os.listdir(dir2) if file.endswith(('.jpg', '.png', '.jpeg'))])

        inter_pairs.append((image1, image2,0))

    return intra_pairs, inter_pairs

# Example usage:
directories = get_cars(base_dir)
num_intra_pairs_per_dir = 5
num_inter_pairs = 10

intra_pairs, inter_pairs = generate_pairs(directories, num_intra_pairs_per_dir, num_inter_pairs)
print("Intra-directory pairs:", len(intra_pairs))
print("Inter-directory pairs:",len(inter_pairs))


100%|██████████| 194112/194112 [06:12<00:00, 520.50it/s]

Intra-directory pairs: 194112

Inter-directory pairs: 194112


In [10]:
# combinations = list(itertools.combinations(imgs, 2))
print(len(intra_pairs))
print(len(inter_pairs))

194112
194112


In [11]:
inter_pairs[:5]

[('../augmented_cars_data/car1 augmented\\IMG_4029.MOV_275.jpg',
  '../augmented_cars_data/car1 orignal\\IMG_4029.MOV_59.jpg',
  0),
 ('../augmented_cars_data/car1 augmented\\IMG_4029.MOV_351.jpg',
  '../augmented_cars_data/car2 augmented\\2332.jpg',
  0),
 ('../augmented_cars_data/car1 orignal\\IMG_4029.MOV_40.jpg',
  '../augmented_cars_data/car1 augmented\\IMG_4029.MOV_233.jpg',
  0),
 ('../augmented_cars_data/car1 orignal\\IMG_4029.MOV_142.jpg',
  '../augmented_cars_data/car1 augmented\\IMG_4029.MOV_381.jpg',
  0),
 ('../augmented_cars_data/car2 augmented\\2347.jpg',
  '../augmented_cars_data/car2 orignal\\2329.jpg',
  0)]

In [12]:
# same_pair = [pair for pair in combinations if pair[0].split('-')[0] == pair[1].split('-')[0]]
# diff_pair = [pair for pair in combinations if pair[0].split('-')[0] != pair[1].split('-')[0]]

In [13]:

with open(f'{mode}-same_pairs.txt', 'w') as fp:
    for pair in intra_pairs:
        fp.write(f'{pair[0]},{pair[1]},{pair[2]}\n')
with open(f'{mode}-diff_pairs.txt', 'w') as fp:
    for pair in inter_pairs:
        fp.write(f'{pair[0]},{pair[1]},{pair[2]}\n')


In [14]:
same_pair = open(f'{mode}-same_pairs.txt','r').readlines()
diff_pair = open(f'{mode}-diff_pairs.txt','r').readlines()
print(same_pair[:5])

['../augmented_cars_data/car1 augmented\\IMG_4029.MOV_1.jpg,../augmented_cars_data/car1 augmented\\IMG_4029.MOV_10.jpg,1\n', '../augmented_cars_data/car1 augmented\\IMG_4029.MOV_1.jpg,../augmented_cars_data/car1 augmented\\IMG_4029.MOV_100.jpg,1\n', '../augmented_cars_data/car1 augmented\\IMG_4029.MOV_1.jpg,../augmented_cars_data/car1 augmented\\IMG_4029.MOV_101.jpg,1\n', '../augmented_cars_data/car1 augmented\\IMG_4029.MOV_1.jpg,../augmented_cars_data/car1 augmented\\IMG_4029.MOV_102.jpg,1\n', '../augmented_cars_data/car1 augmented\\IMG_4029.MOV_1.jpg,../augmented_cars_data/car1 augmented\\IMG_4029.MOV_103.jpg,1\n']


In [15]:
import glob

def get_image_paths(root_dir, extensions=[".jpg", ".jpeg", ".png"]):
    image_paths = []
    
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for file in filenames:
            if any(file.lower().endswith(ext) for ext in extensions):
                image_paths.append(os.path.join(dirpath, file))
    
    return image_paths

# Example usage:
root_directory = '../augmented_cars_data'
image_paths = get_image_paths(root_directory)

# Print the list of image file paths
print(image_paths[:5])

['../augmented_cars_data\\car1 augmented\\IMG_4029.MOV_1.jpg', '../augmented_cars_data\\car1 augmented\\IMG_4029.MOV_10.jpg', '../augmented_cars_data\\car1 augmented\\IMG_4029.MOV_100.jpg', '../augmented_cars_data\\car1 augmented\\IMG_4029.MOV_101.jpg', '../augmented_cars_data\\car1 augmented\\IMG_4029.MOV_102.jpg']


In [16]:
dest_dir = '../augmented_cars_data' + '_embeddings'
dest_dir

'../augmented_cars_data_embeddings'

In [17]:
for img_path in tqdm(image_paths):
    img = cv2.imread(img_path)
    img_embedding =  get_reid_embeddings(cv2.resize(img,(256,256))/255)
    save_path = img_path.replace(base_dir,dest_dir).replace('jpg','npy')
    save_dir = os.path.dirname(save_path)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    np.save(save_path, img_embedding)

100%|██████████| 1152/1152 [01:28<00:00, 12.96it/s]


In [16]:
unique_images = set()
for a,b in new_combined_list:
    unique_images.add(a)
    unique_images.add(b)
len(unique_images)

NameError: name 'new_combined_list' is not defined

In [ ]:
unique_images = list(unique_images)
unique_images[0]

In [ ]:
import cv2
import numpy as np
import random

def augment_image_for_lighting(image):
    # Randomly adjust brightness and contrast
    alpha = random.uniform(0.7, 1.3)  # Adjust brightness
    beta = random.uniform(0.7, 1.3)   # Adjust contrast
    augmented_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    # Randomly apply Gaussian blur
    if random.random() < 0.5:
        blur_kernel_size = random.randint(1, 5) * 2 + 1
        augmented_image = cv2.GaussianBlur(augmented_image, (blur_kernel_size, blur_kernel_size), 0)

    # Randomly add noise
    if random.random() < 0.5:
        noise = np.random.normal(0, 25, image.shape).astype(np.uint8)
        augmented_image = cv2.add(augmented_image, noise)

    return augmented_image

In [ ]:
from tqdm import tqdm

In [ ]:
embedding_map = {}
for img in tqdm(unique_images):
    img_dir,img_name = img.split('-')[0], '-'.join(img.split('-')[1:]) 

    try:
        
        image = cv2.imread(f"{base_dir}/{img_dir}/{img_name}")
    #     print(image)
        image = augment_image_for_lighting(image)
        embedding_map[img] = get_reid_embeddings(cv2.resize(image,(256,256))/255)
    except:
        print(f'count not read {base_dir}/{img_dir}/{img_name}')

In [ ]:
x = []
y = []
count = 1
if not os.path.exists(f'{base_dir}_embeddings'):
    os.mkdir(f'{base_dir}_embeddings')

for pair in tqdm(new_combined_list[2021656:]):
    img1 = pair[0] 
    img2 = pair[1]
    try:
        if img1.endswith('.jpg')  and img2.endswith('jpg'):
            img1_label = pair[0].split('-')[0]
            img2_label = pair[1].split('-')[0]
            img1_embedding = embedding_map[img1]
            img2_embedding = embedding_map[img2]
            embedding = np.concatenate((img1_embedding, img2_embedding), axis=0)
    #         print(embedding.shape)
            label = img2_label==img1_label
            # print(label)
            # print(img2_label, img1_label)
            np.save(f'{base_dir}_embeddings/{count}.npy',np.array([embedding,label]))
            count+=1
    except:
        continue